In [1]:
import os 
import warnings
import time
import numpy as np
import scipy.stats
import sys
import sklearn
import sklearn.datasets

from pyspark.sql import SparkSession
warnings.filterwarnings('ignore')
import pandas as pd

# launch this cell if you have issues on windows with py4j (think about updating your PATH)

os.environ['PYSPARK_DRIVER_PYTHON_OPTS']= "notebook"
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
os.environ['PYSPARK_PYTHON'] = sys.executable

# starts a spark session from notebook

os.environ['PYSPARK_SUBMIT_ARGS'] ="--conf spark.driver.memory=4g  pyspark-shell"


spark = SparkSession \
    .builder \
    .master("local[*]") \
    .config("spark.dynamicAllocation.enabled", "false") \
    .config("spark.executor.instances", "1") \
    .config("spark.executor.cores", "1") \
    .config("spark.executor.memory", "8G") \
    .appName("feature_selection") \
    .getOrCreate()

sc=spark.sparkContext

22/05/21 17:12:27 WARN Utils: Your hostname, pierre-hp resolves to a loopback address: 127.0.1.1; using 192.168.0.194 instead (on interface eno1)
22/05/21 17:12:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/21 17:12:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
train_sessions_engineered = spark.read.csv('../dressipi_recsys2022/session_engineered_features.txt',header=False,
                                          inferSchema=True)

train_purchases = spark.read.load('../dressipi_recsys2022/train_purchases.csv', 
                          format='com.databricks.spark.csv', 
                          header='true', 
                          inferSchema='true')

In [15]:
# sort by session
X = train_sessions_engineered.orderBy('_c0') 
X = X.drop('_c0')
X.take(10)
n_total_features = len(X.columns)

X_np = np.array(X.collect())
print(X_np)
t_X = X_np.transpose()

print(t_X)
B=5
X_RDD = sc.parallelize(t_X,B)

#X_RDD = X.rdd


# Counting the number of rows will allow to implicitly cache the data
print(X_RDD.count())
X_RDD.take(10)

[[3.1200000e+02 3.0000000e+00 5.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [0.0000000e+00 0.0000000e+00 4.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [1.6300000e+02 2.0000000e+00 4.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 ...
 [0.0000000e+00 3.0000000e+00 5.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [4.3600000e+02 3.0000000e+00 2.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 1.4285715e-01]
 [2.5091000e+04 3.0000000e+00 4.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]]
[[3.1200000e+02 0.0000000e+00 1.6300000e+02 ... 0.0000000e+00
  4.3600000e+02 2.5091000e+04]
 [3.0000000e+00 0.0000000e+00 2.0000000e+00 ... 3.0000000e+00
  3.0000000e+00 3.0000000e+00]
 [5.0000000e+00 4.0000000e+00 4.0000000e+00 ... 5.0000000e+00
  2.0000000e+00 4.0000000e+00]
 ...
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 0.0000000e+

22/05/21 17:20:17 WARN TaskSetManager: Stage 48 contains a task of very large size (54692 KiB). The maximum recommended task size is 1000 KiB.


38


22/05/21 17:20:17 WARN TaskSetManager: Stage 49 contains a task of very large size (54692 KiB). The maximum recommended task size is 1000 KiB.
22/05/21 17:20:18 WARN TaskSetManager: Stage 50 contains a task of very large size (54692 KiB). The maximum recommended task size is 1000 KiB.


[array([  312.,     0.,   163., ...,     0.,   436., 25091.]),
 array([3., 0., 2., ..., 3., 3., 3.]),
 array([5., 4., 4., ..., 5., 2., 4.]),
 array([12.,  3.,  8., ..., 11., 11., 10.]),
 array([2020., 2020., 2020., ..., 2020., 2020., 2020.]),
 array([ 9655., 15654., 18316., ..., 25357., 15853., 27400.]),
 array([  312.,    48.,   121., ...,   111.,   132., 24039.]),
 array([ 209.5     ,   48.      ,   94.333336, ...,  111.      ,
          80.71429 , 1337.7368  ]),
 array([ 102.5    ,    0.     ,   36.30733, ...,    0.     ,   38.9788 ,
        5351.505  ]),
 array([ 9.6550e+03, -1.0000e+00, -1.0000e+00, ..., -1.0000e+00,
        -1.0000e+00,  2.5273e+04])]

In [4]:

 # get Y output from the training set sorted by session id
train_purchases_without_date = train_purchases.drop('date')

train_purchases_reformated = train_purchases_without_date.withColumnRenamed("item_id", "item_id_purchased")

Y_order_by_session_id = train_purchases_reformated.drop('session_id')



Y_order_by_session_id.take(10)
Y_numpy = np.array(Y_order_by_session_id.collect())
Y_numpy = Y_numpy.flatten()
print(Y_numpy)

[15085 18626 24911 ... 21630 16962 16631]


In [5]:
X.count() == len(Y_numpy)

True

# Feature Selection

After the feature engineering, we could have a lot of features which are not really worth to give to the predicting model. Considering this problem, we want to use some feature selection algorithms to take the feature which are the most interesting to the model.

We were asked to implement two scalable feature selection algorithms, a ranking algorithm and a forward feature selection.

We could be interested in Minimum-redundancy-maximum-relevance (mRMR) feature selection.


In [6]:
'''
def generate_dataset(n_samples=100, n_informative=1, n_noisy=2, n_redundant=1, random_seed=0):
    """
    generate a dataset to test
    """
    # Set random seed
    np.random.seed(random_seed)
    
    # Use sklearn.datasets.make_regression to generate an artificial dataset where the output y 
    # is correlated with a subset of of the input features
    X, Y = sklearn.datasets.make_classification(n_samples=n_samples, 
                                            n_features=n_informative+n_noisy, 
                                            n_informative=n_informative)
    
    # Create a random mixing matrix for generating redundant features from informative ones
    mixing_matrix = np.random.random((n_informative, n_redundant))
    
    # Create redundant features by taking random linear combinations of informative features
    redundant_features = np.dot(X[:,0:n_informative], mixing_matrix)
    
    # Add redundant features to the input data
    X = np.concatenate((X, redundant_features), axis=1)
    
    # Return input data X, output data Y
    return X, Y


# Let us generate the dataset
N = 1000
n_informative = 100
n_noisy = 100
n_redundant = 100

X, Y = generate_dataset(n_samples=N, 
                        n_informative=n_informative, 
                        n_noisy=n_noisy, 
                        n_redundant=n_redundant)

t_X = np.transpose(X)

B=3
X_RDD=sc.parallelize(t_X,B).cache()

# Counting the number of rows will allow to implicitly cache the data
X_RDD.count()
X_RDD.take(10)

print(type(Y))'''

'\ndef generate_dataset(n_samples=100, n_informative=1, n_noisy=2, n_redundant=1, random_seed=0):\n    """\n    generate a dataset to test\n    """\n    # Set random seed\n    np.random.seed(random_seed)\n    \n    # Use sklearn.datasets.make_regression to generate an artificial dataset where the output y \n    # is correlated with a subset of of the input features\n    X, Y = sklearn.datasets.make_classification(n_samples=n_samples, \n                                            n_features=n_informative+n_noisy, \n                                            n_informative=n_informative)\n    \n    # Create a random mixing matrix for generating redundant features from informative ones\n    mixing_matrix = np.random.random((n_informative, n_redundant))\n    \n    # Create redundant features by taking random linear combinations of informative features\n    redundant_features = np.dot(X[:,0:n_informative], mixing_matrix)\n    \n    # Add redundant features to the input data\n    X = np.conc

In [7]:
def get_score_mrmr(x, y):
    """
    return the correlation value between two variable in absolute value
    """

    return np.abs(scipy.stats.pearsonr(x, y)[0])

In [8]:
def get_mrmr_score_spark(x,y,selected_features):
    """
    x : the feature to evaluate to add into the model
    y : output value
    selected_features : the features already selected
    return the score for x with the rest of selected variables
    """
    
    # Get correlation score between feature x and output y (relevance)
    score_x_y_s = get_score_mrmr(x, y)
    
    nb_selected_features = selected_features.shape[0]
    # If some features have already been selected
    if nb_selected_features > 0:
        
        # Get corrrelation scores between x and each feature already selected (redundancy)
        score_features_x_s = np.zeros(nb_selected_features, dtype=float)
        
        for j in range(nb_selected_features):
                
            score_x_s_j = get_score_mrmr(x, selected_features[j,:])
                
            score_features_x_s[j] = score_x_s_j
                
        # Final score is relevance to output feature - average redundancy with already selected features
        score_x_y_s = score_x_y_s - np.mean(score_features_x_s)
        
    return score_x_y_s

In [9]:
def mrmr_spark(n_total_features, K, sc, X_RDD, Y):
    """
    n_total_features : number of total features
    K : number of feature to select
    sc : spark context
    X_RDD : RDD of the variable X
    Y: Output data
    
    return the indice of selected features and time execution using mrmr
    """
    time_execution = []
    remaining_features_indices = list(range(n_total_features))
    selected_features_indices = []

    for k in range(K):
        print("Step: "+str(k))
    
        start_time=time.time()
        # Get the subset of selected features values, and cast as an array
        selected_features = X_RDD.zipWithIndex().filter(lambda x: x[1] in selected_features_indices).map(lambda x: x[0]).collect()
        selected_features = np.array(selected_features)
    
        # mRMR scores are computed by first filtering `t_X` to remove already selected features, and then mapping 
        # each remaining feature using the `get_mrmr_score_spark` function
        scores = X_RDD.zipWithIndex().filter(lambda x: x[1] in remaining_features_indices).map(lambda x:get_mrmr_score_spark(x[0],Y,selected_features)).collect()
    
        # Once all mRMR scores are computed, the index of the feature with the highest score is selected
        scores = np.array(scores)
    
        index_max_score_features = np.argmax(scores)
    
        selected_features_indices.append(remaining_features_indices[index_max_score_features])
    
        del(remaining_features_indices[index_max_score_features])
    
        print(time.time()-start_time)
        time_execution.append(time.time()-start_time)
        
    return selected_features_indices, time_execution

In [16]:
selected_features_indices, execution_time = mrmr_spark(n_total_features, 5, sc,X_RDD, Y_numpy)

Step: 0


22/05/21 17:20:34 WARN TaskSetManager: Stage 51 contains a task of very large size (54692 KiB). The maximum recommended task size is 1000 KiB.
22/05/21 17:20:35 WARN TaskSetManager: Stage 52 contains a task of very large size (54692 KiB). The maximum recommended task size is 1000 KiB.
22/05/21 17:20:35 WARN TaskSetManager: Stage 53 contains a task of very large size (54692 KiB). The maximum recommended task size is 1000 KiB.
22/05/21 17:20:36 WARN TaskSetManager: Stage 54 contains a task of very large size (54692 KiB). The maximum recommended task size is 1000 KiB.
/home/pdefraene/.local/lib/python3.10/site-packages/scipy/stats/_stats_py.py:4068: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())
22/05/21 17:20:37 WARN TaskSetManager: Stage 55 contains a task of very large size (54692 KiB). The maximum recommended task size is 1000 KiB.


3.7156221866607666
Step: 1


22/05/21 17:20:38 WARN TaskSetManager: Stage 56 contains a task of very large size (54692 KiB). The maximum recommended task size is 1000 KiB.
22/05/21 17:20:39 WARN TaskSetManager: Stage 57 contains a task of very large size (54692 KiB). The maximum recommended task size is 1000 KiB.
22/05/21 17:20:39 WARN TaskSetManager: Stage 58 contains a task of very large size (54692 KiB). The maximum recommended task size is 1000 KiB.
/home/pdefraene/.local/lib/python3.10/site-packages/scipy/stats/_stats_py.py:4068: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/home/pdefraene/.local/lib/python3.10/site-packages/scipy/stats/_stats_py.py:4068: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/home/pdefraene/.local/lib/python3.10/site-packages/scipy/stats/_stats_py.py:4068: PearsonRConstantInputW

3.279282569885254
Step: 2


...

3.9825942516326904
Step: 3


22/05/21 17:20:45 WARN TaskSetManager: Stage 64 contains a task of very large size (54692 KiB). The maximum recommended task size is 1000 KiB.
22/05/21 17:20:46 WARN TaskSetManager: Stage 65 contains a task of very large size (54692 KiB). The maximum recommended task size is 1000 KiB.
22/05/21 17:20:47 WARN TaskSetManager: Stage 66 contains a task of very large size (54692 KiB). The maximum recommended task size is 1000 KiB.


4.119117259979248
Step: 4


22/05/21 17:20:49 WARN TaskSetManager: Stage 67 contains a task of very large size (54692 KiB). The maximum recommended task size is 1000 KiB.
22/05/21 17:20:49 WARN TaskSetManager: Stage 68 contains a task of very large size (54692 KiB). The maximum recommended task size is 1000 KiB.
22/05/21 17:20:50 WARN TaskSetManager: Stage 69 contains a task of very large size (54692 KiB). The maximum recommended task size is 1000 KiB.
22/05/21 17:20:51 WARN TaskSetManager: Stage 70 contains a task of very large size (54692 KiB). The maximum recommended task size is 1000 KiB.


4.917078733444214


In [17]:
selected_features_indices

[36, 0, 1, 2, 3]

Then, we will look a the forward feature Selection by training on a decision tree with every feature and add the features which get the best results.

In [12]:
from sklearn import tree
from sklearn.model_selection import train_test_split

def fit_get_score(x,y,selected_features):
    """
    train an model to get the score with the addition of a specified x feature
    """    
    
    n_selected_features = selected_features.shape[0]
    
    if n_selected_features>0:
        # need to merge x and the already selected features
        x = np.vstack((selected_features,x))
        x = x.transpose()
    else:
        x = x.reshape(-1, 1)
    
    #split data
    x_train,x_test,y_train,y_test = train_test_split(x,y, test_size= 0.3)
    
    # train on training data
    dt = tree.DecisionTreeClassifier()
    dt = dt.fit(x_train,y_train)
    
    # get score on testing set
    return dt.score(x_test, y_test)

In [13]:
def forward_feature_selection(n_total_features, K, sc, X_RDD, Y):
    """
    n_total_features : number of total features
    K : number of feature to select
    sc : spark context
    X_RDD : RDD of the variable X
    Y: Output data
    
    return the indice of selected features and time execution
    by using a decision tree as model to calculate the score
    """
    time_execution = []
    
    remaining_features_indices = list(range(n_total_features))
    selected_features_indices = []
    
    for k in range(K):
        print("Step: "+str(k))
    
        start_time=time.time()

        # Get the subset of selected features values, and cast as an array
        selected_features = X_RDD.zipWithIndex().filter(lambda x: x[1] in selected_features_indices).map(lambda x: x[0]).collect()
        selected_features = np.array(selected_features)
        
    
        #  scores for a certain model are computed by first filtering `t_X` to remove already selected features, and then mapping 
        # each remaining feature using the `fit_get_score` function
        scores = X_RDD.zipWithIndex().filter(lambda x: x[1] in remaining_features_indices).map(lambda x:fit_get_score(x[0],Y,selected_features)).collect()
    
        # Once all scores are computed, the index of the feature with the highest value is chosen
        scores = np.array(scores)
        
        print("best_score :", np.max(scores))
    
        index_max_score_features = np.argmax(scores)
    
        selected_features_indices.append(remaining_features_indices[index_max_score_features])
    
        del(remaining_features_indices[index_max_score_features])
    
        print(time.time()-start_time)
        time_execution.append(time.time()-start_time)
        
    return selected_features_indices, time_execution
    

In [14]:
selected_features_indices_forward, execution_time_forward = forward_feature_selection(n_total_features, 10, sc,X_RDD, Y_numpy)
selected_features_indices_forward

Step: 0


22/05/21 17:13:36 WARN TaskSetManager: Stage 39 contains a task of very large size (54692 KiB). The maximum recommended task size is 1000 KiB.
22/05/21 17:13:36 WARN TaskSetManager: Stage 40 contains a task of very large size (54692 KiB). The maximum recommended task size is 1000 KiB.
22/05/21 17:13:37 WARN TaskSetManager: Stage 41 contains a task of very large size (54692 KiB). The maximum recommended task size is 1000 KiB.
22/05/21 17:13:38 WARN TaskSetManager: Stage 42 contains a task of very large size (54692 KiB). The maximum recommended task size is 1000 KiB.
22/05/21 17:13:39 ERROR Executor: Exception in task 3.0 in stage 42.0 (TID 203)]
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/pdefraene/Downloads/spark-3.2.1-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/worker.py", line 619, in main
    process()
  File "/home/pdefraene/Downloads/spark-3.2.1-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/worker.py", line 611, in process
    seri

22/05/21 17:13:40 WARN TaskSetManager: Lost task 2.0 in stage 42.0 (TID 202) (192.168.0.194 executor driver): TaskKilled (Stage cancelled)
22/05/21 17:13:40 WARN TaskSetManager: Lost task 4.0 in stage 42.0 (TID 204) (192.168.0.194 executor driver): TaskKilled (Stage cancelled)


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 42.0 failed 1 times, most recent failure: Lost task 3.0 in stage 42.0 (TID 203) (192.168.0.194 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/pdefraene/Downloads/spark-3.2.1-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/worker.py", line 619, in main
    process()
  File "/home/pdefraene/Downloads/spark-3.2.1-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/worker.py", line 611, in process
    serializer.dump_stream(out_iter, outfile)
  File "/home/pdefraene/Downloads/spark-3.2.1-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/serializers.py", line 259, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/home/pdefraene/Downloads/spark-3.2.1-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/util.py", line 74, in wrapper
    return

In [ ]:
def forward_feature_selection_diff(n_total_features, inc_accuracy, sc, X_RDD, Y):
    """
    n_total_features : number of total features
    inc_accuracy : increase of accuracy needed to continue the algorithm
    sc : spark context
    X_RDD : RDD of the variable X
    Y: Output data
    
    return the indice of selected features and time execution
    by using a decision tree as model to calculate the score
    """
    time_execution = []
    
    remaining_features_indices = list(range(n_total_features))
    selected_features_indices = []
    
    last_best_score = 0
    diff_accuracy = 1
    k = 0
    while diff_accuracy > inc_accuracy:
        print("Step: "+str(k))
    
        start_time=time.time()

        # Get the subset of selected features values, and cast as an array
        selected_features = X_RDD.zipWithIndex().filter(lambda x: x[1] in selected_features_indices).map(lambda x: x[0]).collect()
        selected_features = np.array(selected_features)
        
    
        #  scores for a certain model are computed by first filtering `t_X` to remove already selected features, and then mapping 
        # each remaining feature using the `fit_get_score` function
        scores = X_RDD.zipWithIndex().filter(lambda x: x[1] in remaining_features_indices).map(lambda x:fit_get_score(x[0],Y,selected_features)).collect()
    
        # Once all scores are computed, the index of the feature with the highest value is chosen
        scores = np.array(scores)
        
        # compute the difference between last result and new result
        best_score = np.max(scores)
        diff_accuracy = best_score - last_best_score
        
        
        print("best_accuracy:", best_score)
        
        if best_score > last_best_score:
        
            index_max_score_features = np.argmax(scores)
    
            selected_features_indices.append(remaining_features_indices[index_max_score_features])
    
            del(remaining_features_indices[index_max_score_features])
        
        last_best_score = best_score
        print(time.time()-start_time)
        time_execution.append(time.time()-start_time)
        
        k += 1
        
    return selected_features_indices, time_execution

In [ ]:
selected_features_indices_forward_diff, execution_time_forward_diff = forward_feature_selection_diff(n_total_features, 0.001, sc,X_RDD, Y_numpy)
selected_features_indices_forward_diff

In [ ]:
#spark.stop()

22/05/21 17:13:42 WARN PythonRunner: Incomplete task 0.0 in stage 42 (TID 200) interrupted: Attempting to kill Python Worker
22/05/21 17:13:42 WARN PythonRunner: Incomplete task 1.0 in stage 42 (TID 201) interrupted: Attempting to kill Python Worker
22/05/21 17:13:42 WARN TaskSetManager: Lost task 0.0 in stage 42.0 (TID 200) (192.168.0.194 executor driver): TaskKilled (Stage cancelled)
22/05/21 17:13:42 WARN TaskSetManager: Lost task 1.0 in stage 42.0 (TID 201) (192.168.0.194 executor driver): TaskKilled (Stage cancelled)
